In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
###Document Structure

from langchain_core.documents import Document

In [ ]:
doc=Document(
    page_content="this is the main text content I am using to create RAG",
    metadata={
        "source":"exmaple.txt",
        "pages":1,
        "author":"Krish Naik",
        "date_created":"2025-01-01"
    }
)
doc

Document(metadata={'source': 'exmaple.txt', 'pages': 1, 'author': 'Krish Naik', 'date_created': '2025-01-01'}, page_content='this is the main text content I am using to create RAG')

In [ ]:
## Create a simple txt file
import os
os.makedirs("/content/drive/MyDrive/AI Engineering/Traditional RAG/data/text_files",exist_ok=True)

In [ ]:
sample_texts={
    "/content/drive/MyDrive/AI Engineering/Traditional RAG/data/text_files/python_intro.txt":"""Python Programming Introduction

Python is a high-level, interpreted programming language known for its simplicity and readability.
Created by Guido van Rossum and first released in 1991, Python has become one of the most popular
programming languages in the world.

Key Features:
- Easy to learn and use
- Extensive standard library
- Cross-platform compatibility
- Strong community support

Python is widely used in web development, data science, artificial intelligence, and automation.""",

    "../data/text_files/machine_learning.txt": """Machine Learning Basics

Machine learning is a subset of artificial intelligence that enables systems to learn and improve
from experience without being explicitly programmed. It focuses on developing computer programs
that can access data and use it to learn for themselves.

Types of Machine Learning:
1. Supervised Learning: Learning with labeled data
2. Unsupervised Learning: Finding patterns in unlabeled data
3. Reinforcement Learning: Learning through rewards and penalties

Applications include image recognition, speech processing, and recommendation systems


    """

}

for filepath,content in sample_texts.items():
    with open(filepath,'w',encoding="utf-8") as f:
        f.write(content)

print("✅ Sample text files created!")

✅ Sample text files created!


In [ ]:
### TextLoader
from langchain.document_loaders import TextLoader

from langchain_community.document_loaders import TextLoader

loader=TextLoader("/content/drive/MyDrive/AI Engineering/Traditional RAG/data/text_files/python_intro.txt",encoding="utf-8")
document=loader.load()
print(document)

[Document(metadata={'source': '/content/drive/MyDrive/AI Engineering/Traditional RAG/data/text_files/python_intro.txt'}, page_content='Python Programming Introduction\n\nPython is a high-level, interpreted programming language known for its simplicity and readability.\nCreated by Guido van Rossum and first released in 1991, Python has become one of the most popular\nprogramming languages in the world.\n\nKey Features:\n- Easy to learn and use\n- Extensive standard library\n- Cross-platform compatibility\n- Strong community support\n\nPython is widely used in web development, data science, artificial intelligence, and automation.')]


In [ ]:
### Directory Loader
from langchain_community.document_loaders import DirectoryLoader

## load all the text files from the directory
dir_loader=DirectoryLoader(
    "/content/drive/MyDrive/AI Engineering/Traditional RAG/data/text_files",
    glob="**/*.txt", ## Pattern to match files
    loader_cls= TextLoader, ##loader class to use
    loader_kwargs={'encoding': 'utf-8'},
    show_progress=False

)

documents=dir_loader.load()
documents

[Document(metadata={'source': '/content/drive/MyDrive/AI Engineering/Traditional RAG/data/text_files/python_intro.txt'}, page_content='Python Programming Introduction\n\nPython is a high-level, interpreted programming language known for its simplicity and readability.\nCreated by Guido van Rossum and first released in 1991, Python has become one of the most popular\nprogramming languages in the world.\n\nKey Features:\n- Easy to learn and use\n- Extensive standard library\n- Cross-platform compatibility\n- Strong community support\n\nPython is widely used in web development, data science, artificial intelligence, and automation.')]

In [ ]:
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 75.7 MB/s eta 0:00:00


In [ ]:
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader

## load all the text files from the directory
dir_loader=DirectoryLoader(
    "/content/drive/MyDrive/AI Engineering/Traditional RAG/data",
    glob="**/*.pdf", ## Pattern to match files
    loader_cls= PyMuPDFLoader, ##loader class to use
    show_progress=False

)

pdf_documents=dir_loader.load()
pdf_documents

[Document(metadata={'producer': 'pdfTeX-1.40.17', 'creator': 'LaTeX with hyperref package', 'creationdate': '2019-06-05T00:42:23+00:00', 'source': '/content/drive/MyDrive/AI Engineering/Traditional RAG/data/MELD Dataset.pdf', 'file_path': '/content/drive/MyDrive/AI Engineering/Traditional RAG/data/MELD Dataset.pdf', 'total_pages': 10, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2019-06-05T00:42:23+00:00', 'trapped': '', 'modDate': 'D:20190605004223Z', 'creationDate': 'D:20190605004223Z', 'page': 0}, page_content='MELD: A Multimodal Multi-Party Dataset\nfor Emotion Recognition in Conversations\nSoujanya Poria†, Devamanyu HazarikaΦ, Navonil Majumder‡,\nGautam Naik¶, Erik Cambria¶, Rada Mihalceaι\n†Information Systems Technology and Design, SUTD, Singapore\nΦSchool of Computing, National University of Singapore, Singapore\n‡Centro de Investigaci´on en Computaci´on, Instituto Polit´ecnico Nacional, Mexico\n¶Computer Science & Engineering, Nany

In [ ]:
type(pdf_documents[0])

langchain_core.documents.base.Document

### RAG Pipelines- Data Ingestion to Vector DB Pipeline

In [ ]:
!pip install chromadb

In [ ]:
!pip install langchain_community

In [ ]:
!pip install pypdf

In [ ]:
import os
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path

In [ ]:
### Read all the pdf's inside the directory
def process_all_pdfs(pdf_directory):
    """Process all PDF files in a directory"""
    all_documents = []
    pdf_dir = Path(pdf_directory)

    # Find all PDF files recursively
    pdf_files = list(pdf_dir.glob("**/*.pdf"))

    print(f"Found {len(pdf_files)} PDF files to process")

    for pdf_file in pdf_files:
        print(f"\nProcessing: {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()

            # Add source information to metadata
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'

            all_documents.extend(documents)
            print(f"  ✓ Loaded {len(documents)} pages")

        except Exception as e:
            print(f"  ✗ Error: {e}")

    print(f"\nTotal documents loaded: {len(all_documents)}")
    return all_documents

# Process all PDFs in the data directory
all_pdf_documents = process_all_pdfs("/content/drive/MyDrive/AI Engineering/Traditional RAG/data/pdf")

Found 1 PDF files to process

Processing: AoM.pdf
  ✓ Loaded 11 pages

Total documents loaded: 11


In [ ]:
all_pdf_documents

[Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-06-05T01:00:14+00:00', 'author': '', 'keywords': '', 'moddate': '2023-06-05T01:00:14+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': '/content/drive/MyDrive/AI Engineering/Traditional RAG/data/pdf/AoM.pdf', 'total_pages': 11, 'page': 0, 'page_label': '1', 'source_file': 'AoM.pdf', 'file_type': 'pdf'}, page_content='AoM: Detecting Aspect-oriented Information for Multimodal\nAspect-Based Sentiment Analysis\nRu Zhou1 Wenya Guo1∗ Xumeng Liu1 Shenglong Yu1\nYing Zhang1 Xiaojie Yuan1\n1 College of Computer Science, TKLNDST, Nankai University, Tianjin, China\n{zhouru,guowenya,liuxumeng,yushenglong,zhangying}@dbis.nankai.edu.cn\nyuanxj@nankai.edu.cn\nAbstract\nMultimodal aspect-based sentiment analysis\n(MABSA) aims to extract aspects from text-\nimage pairs and r

In [ ]:
### Text splitting get into chunks

def split_documents(documents,chunk_size=1000,chunk_overlap=200):
    """Split documents into smaller chunks for better RAG performance"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")

    # Show example of a chunk
    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")

    return split_docs

In [ ]:
chunks=split_documents(all_pdf_documents)
chunks

Split 11 documents into 57 chunks

Example chunk:
Content: AoM: Detecting Aspect-oriented Information for Multimodal
Aspect-Based Sentiment Analysis
Ru Zhou1 Wenya Guo1∗ Xumeng Liu1 Shenglong Yu1
Ying Zhang1 Xiaojie Yuan1
1 College of Computer Science, TKLNDS...
Metadata: {'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-06-05T01:00:14+00:00', 'author': '', 'keywords': '', 'moddate': '2023-06-05T01:00:14+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': '/content/drive/MyDrive/AI Engineering/Traditional RAG/data/pdf/AoM.pdf', 'total_pages': 11, 'page': 0, 'page_label': '1', 'source_file': 'AoM.pdf', 'file_type': 'pdf'}


[Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-06-05T01:00:14+00:00', 'author': '', 'keywords': '', 'moddate': '2023-06-05T01:00:14+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': '/content/drive/MyDrive/AI Engineering/Traditional RAG/data/pdf/AoM.pdf', 'total_pages': 11, 'page': 0, 'page_label': '1', 'source_file': 'AoM.pdf', 'file_type': 'pdf'}, page_content='AoM: Detecting Aspect-oriented Information for Multimodal\nAspect-Based Sentiment Analysis\nRu Zhou1 Wenya Guo1∗ Xumeng Liu1 Shenglong Yu1\nYing Zhang1 Xiaojie Yuan1\n1 College of Computer Science, TKLNDST, Nankai University, Tianjin, China\n{zhouru,guowenya,liuxumeng,yushenglong,zhangying}@dbis.nankai.edu.cn\nyuanxj@nankai.edu.cn\nAbstract\nMultimodal aspect-based sentiment analysis\n(MABSA) aims to extract aspects from text-\nimage pairs and r

### embedding And vectorStoreDB

In [ ]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
class EmbeddingManager:
    """Handles document embedding generation using SentenceTransformer"""

    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
        Initialize the embedding manager

        Args:
            model_name: HuggingFace model name for sentence embeddings
        """
        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
        """Load the SentenceTransformer model"""
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generate embeddings for a list of texts

        Args:
            texts: List of text strings to embed

        Returns:
            numpy array of embeddings with shape (len(texts), embedding_dim)
        """
        if not self.model:
            raise ValueError("Model not loaded")

        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings


## initialize the embedding manager

embedding_manager=EmbeddingManager()
embedding_manager

Loading embedding model: all-MiniLM-L6-v2
Model loaded successfully. Embedding dimension: 384


### VectorStore

In [ ]:
import os

### Correct store embedding in RAM

In [ ]:
import os
import uuid
import numpy as np
import chromadb
from typing import List, Any


class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""

    def __init__(
        self,
        collection_name: str = "pdf_documents",
        persist_directory: str = "/content/drive/MyDrive/AI Engineering/Traditional RAG/vector_store"
    ):
        """
        Initialize the vector store

        Args:
            collection_name: Name of the ChromaDB collection
            persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            # Ensure persistence dir exists
            os.makedirs(self.persist_directory, exist_ok=True)

            print("👉 Trying to initialize persistent ChromaDB...")
            self.client = chromadb.PersistentClient(path=self.persist_directory)

            # Create or get collection
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "PDF document embeddings for RAG"}
            )
            print(f"✅ Persistent vector store initialized. Collection: {self.collection_name}")
            print(f"   Existing documents in collection: {self.collection.count()}")

        except Exception as e:
            print(f"⚠️ Persistent DB failed: {e}")
            print("👉 Falling back to in-memory ChromaDB (data will NOT be saved).")

            # Fallback to in-memory
            self.client = chromadb.Client()
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "In-memory PDF document embeddings for RAG"}
            )
            print(f"✅ In-memory vector store initialized. Collection: {self.collection_name}")

    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """
        Add documents and their embeddings to the vector store

        Args:
            documents: List of LangChain documents
            embeddings: Corresponding embeddings for the documents
        """
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")

        print(f"Adding {len(documents)} documents to vector store...")

        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []

        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)

            metadata = dict(doc.metadata)
            metadata["doc_index"] = i
            metadata["content_length"] = len(doc.page_content)
            metadatas.append(metadata)

            documents_text.append(doc.page_content)
            embeddings_list.append(embedding.tolist())

        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_text,
            )
            print(f"✅ Successfully added {len(documents)} documents to vector store")
            print(f"   Total documents in collection: {self.collection.count()}")

        except Exception as e:
            print(f"❌ Error adding documents to vector store: {e}")
            raise


# ---- Initialize ----
vectorstore = VectorStore()
vectorstore

👉 Trying to initialize persistent ChromaDB...
⚠️ Persistent DB failed: Database error: error returned from database: (code: 1) no such table: tenants
👉 Falling back to in-memory ChromaDB (data will NOT be saved).
✅ In-memory vector store initialized. Collection: pdf_documents


In [ ]:
texts=[doc.page_content for doc in chunks]
texts

['AoM: Detecting Aspect-oriented Information for Multimodal\nAspect-Based Sentiment Analysis\nRu Zhou1 Wenya Guo1∗ Xumeng Liu1 Shenglong Yu1\nYing Zhang1 Xiaojie Yuan1\n1 College of Computer Science, TKLNDST, Nankai University, Tianjin, China\n{zhouru,guowenya,liuxumeng,yushenglong,zhangying}@dbis.nankai.edu.cn\nyuanxj@nankai.edu.cn\nAbstract\nMultimodal aspect-based sentiment analysis\n(MABSA) aims to extract aspects from text-\nimage pairs and recognize their sentiments. Ex-\nisting methods make great efforts to align the\nwhole image to corresponding aspects. How-\never, different regions of the image may relate\nto different aspects in the same sentence, and\ncoarsely establishing image-aspect alignment\nwill introduce noise to aspect-based sentiment\nanalysis (i.e., visual noise). Besides, the sen-\ntiment of a specific aspect can also be inter-\nfered by descriptions of other aspects (i.e., tex-\ntual noise). Considering the aforementioned\nnoises, this paper proposes an Aspect-o

In [ ]:
### Convert the text to embeddings
texts=[doc.page_content for doc in chunks]

## Generate the Embeddings

embeddings=embedding_manager.generate_embeddings(texts)

##store int he vector dtaabase
vectorstore.add_documents(chunks,embeddings)

Generating embeddings for 57 texts...


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Generated embeddings with shape: (57, 384)
Adding 57 documents to vector store...
✅ Successfully added 57 documents to vector store
   Total documents in collection: 114


### Retriever Pipeline From VectorStore

In [ ]:
class RAGRetriever:
    """Handles query-based retrieval from the vector store"""

    def __init__(self, vector_store: VectorStore, embedding_manager: EmbeddingManager):
        """
        Initialize the retriever

        Args:
            vector_store: Vector store containing document embeddings
            embedding_manager: Manager for generating query embeddings
        """
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager

    def retrieve(self, query: str, top_k: int = 5, score_threshold: float = 0.0) -> List[Dict[str, Any]]:
        """
        Retrieve relevant documents for a query

        Args:
            query: The search query
            top_k: Number of top results to return
            score_threshold: Minimum similarity score threshold

        Returns:
            List of dictionaries containing retrieved documents and metadata
        """
        print(f"Retrieving documents for query: '{query}'")
        print(f"Top K: {top_k}, Score threshold: {score_threshold}")

        # Generate query embedding
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]

        # Search in vector store
        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )

            # Process results
            retrieved_docs = []

            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]

                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    # Convert distance to similarity score (ChromaDB uses cosine distance)
                    similarity_score = 1 - distance

                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                            'id': doc_id,
                            'content': document,
                            'metadata': metadata,
                            'similarity_score': similarity_score,
                            'distance': distance,
                            'rank': i + 1
                        })

                print(f"Retrieved {len(retrieved_docs)} documents (after filtering)")
            else:
                print("No documents found")

            return retrieved_docs

        except Exception as e:
            print(f"Error during retrieval: {e}")
            return []

rag_retriever=RAGRetriever(vectorstore,embedding_manager)



In [ ]:
rag_retriever

In [ ]:
rag_retriever.retrieve("what multimodal aspect-based sentiment analysis ")

Retrieving documents for query: 'what multimodal aspect-based sentiment analysis '
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generated embeddings with shape: (1, 384)
Retrieved 5 documents (after filtering)


[{'id': 'doc_0edf29b3_0',
  'content': 'AoM: Detecting Aspect-oriented Information for Multimodal\nAspect-Based Sentiment Analysis\nRu Zhou1 Wenya Guo1∗ Xumeng Liu1 Shenglong Yu1\nYing Zhang1 Xiaojie Yuan1\n1 College of Computer Science, TKLNDST, Nankai University, Tianjin, China\n{zhouru,guowenya,liuxumeng,yushenglong,zhangying}@dbis.nankai.edu.cn\nyuanxj@nankai.edu.cn\nAbstract\nMultimodal aspect-based sentiment analysis\n(MABSA) aims to extract aspects from text-\nimage pairs and recognize their sentiments. Ex-\nisting methods make great efforts to align the\nwhole image to corresponding aspects. How-\never, different regions of the image may relate\nto different aspects in the same sentence, and\ncoarsely establishing image-aspect alignment\nwill introduce noise to aspect-based sentiment\nanalysis (i.e., visual noise). Besides, the sen-\ntiment of a specific aspect can also be inter-\nfered by descriptions of other aspects (i.e., tex-\ntual noise). Considering the aforementioned\nn

In [ ]:
rag_retriever.retrieve("Which datasets used for  for multimodal aspect-based sentiment analysis")

Retrieving documents for query: 'Which datasets used for  for multimodal aspect-based sentiment analysis'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generated embeddings with shape: (1, 384)
Retrieved 5 documents (after filtering)


[{'id': 'doc_0edf29b3_0',
  'content': 'AoM: Detecting Aspect-oriented Information for Multimodal\nAspect-Based Sentiment Analysis\nRu Zhou1 Wenya Guo1∗ Xumeng Liu1 Shenglong Yu1\nYing Zhang1 Xiaojie Yuan1\n1 College of Computer Science, TKLNDST, Nankai University, Tianjin, China\n{zhouru,guowenya,liuxumeng,yushenglong,zhangying}@dbis.nankai.edu.cn\nyuanxj@nankai.edu.cn\nAbstract\nMultimodal aspect-based sentiment analysis\n(MABSA) aims to extract aspects from text-\nimage pairs and recognize their sentiments. Ex-\nisting methods make great efforts to align the\nwhole image to corresponding aspects. How-\never, different regions of the image may relate\nto different aspects in the same sentence, and\ncoarsely establishing image-aspect alignment\nwill introduce noise to aspect-based sentiment\nanalysis (i.e., visual noise). Besides, the sen-\ntiment of a specific aspect can also be inter-\nfered by descriptions of other aspects (i.e., tex-\ntual noise). Considering the aforementioned\nn

### RAG Pipeline- VectorDB To LLM Output Generation

In [ ]:
import os
os.environ["GROQ_API_KEY"] = "GROQ_API_KEY"

print(os.getenv("GROQ_API_KEY"))

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


In [ ]:
!pip install langchain_groq

In [ ]:
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.schema import HumanMessage, SystemMessage

In [ ]:
class GroqLLM:
    def __init__(self, model_name: str = "gemma2-9b-it", api_key: str =None):
        """
        Initialize Groq LLM

        Args:
            model_name: Groq model name (qwen2-72b-instruct, llama3-70b-8192, etc.)
            api_key: Groq API key (or set GROQ_API_KEY environment variable)
        """
        self.model_name = model_name
        self.api_key = api_key or os.environ.get("GROQ_API_KEY")

        if not self.api_key:
            raise ValueError("Groq API key is required. Set GROQ_API_KEY environment variable or pass api_key parameter.")

        self.llm = ChatGroq(
            groq_api_key=self.api_key,
            model_name=self.model_name,
            temperature=0.1,
            max_tokens=1024
        )

        print(f"Initialized Groq LLM with model: {self.model_name}")

    def generate_response(self, query: str, context: str, max_length: int = 500) -> str:
        """
        Generate response using retrieved context

        Args:
            query: User question
            context: Retrieved document context
            max_length: Maximum response length

        Returns:
            Generated response string
        """

        # Create prompt template
        prompt_template = PromptTemplate(
            input_variables=["context", "question"],
            template="""You are a helpful AI assistant. Use the following context to answer the question accurately and concisely.

Context:
{context}

Question: {question}

Answer: Provide a clear and informative answer based on the context above. If the context doesn't contain enough information to answer the question, say so."""
        )

        # Format the prompt
        formatted_prompt = prompt_template.format(context=context, question=query)

        try:
            # Generate response
            messages = [HumanMessage(content=formatted_prompt)]
            response = self.llm.invoke(messages)
            return response.content

        except Exception as e:
            return f"Error generating response: {str(e)}"

    def generate_response_simple(self, query: str, context: str) -> str:
        """
        Simple response generation without complex prompting

        Args:
            query: User question
            context: Retrieved context

        Returns:
            Generated response
        """
        simple_prompt = f"""Based on this context: {context}

Question: {query}

Answer:"""

        try:
            messages = [HumanMessage(content=simple_prompt)]
            response = self.llm.invoke(messages)
            return response.content
        except Exception as e:
            return f"Error: {str(e)}"



In [ ]:
# Initialize Groq LLM (you'll need to set GROQ_API_KEY environment variable)
try:
    groq_llm = GroqLLM(api_key=os.getenv("GROQ_API_KEY"))
    print("Groq LLM initialized successfully!")
except ValueError as e:
    print(f"Warning: {e}")
    print("Please set your GROQ_API_KEY environment variable to use the LLM.")
    groq_llm = None

Initialized Groq LLM with model: gemma2-9b-it
Groq LLM initialized successfully!


In [ ]:
# Initialize Groq LLM (you'll need to set GROQ_API_KEY environment variable)
try:
    groq_llm = GroqLLM(api_key=os.getenv("GROQ_API_KEY"))
    print("Groq LLM initialized successfully!")
except ValueError as e:
    print(f"Warning: {e}")
    print("Please set your GROQ_API_KEY environment variable to use the LLM.")
    groq_llm = None

Initialized Groq LLM with model: gemma2-9b-it
Groq LLM initialized successfully!


In [ ]:
### get the context from the retriever and pass it to the LLM

rag_retriever.retrieve("what is multimodal aspect-based sentiment analysis")

Retrieving documents for query: 'what is multimodal aspect-based sentiment analysis'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generated embeddings with shape: (1, 384)
Retrieved 5 documents (after filtering)


[{'id': 'doc_0edf29b3_0',
  'content': 'AoM: Detecting Aspect-oriented Information for Multimodal\nAspect-Based Sentiment Analysis\nRu Zhou1 Wenya Guo1∗ Xumeng Liu1 Shenglong Yu1\nYing Zhang1 Xiaojie Yuan1\n1 College of Computer Science, TKLNDST, Nankai University, Tianjin, China\n{zhouru,guowenya,liuxumeng,yushenglong,zhangying}@dbis.nankai.edu.cn\nyuanxj@nankai.edu.cn\nAbstract\nMultimodal aspect-based sentiment analysis\n(MABSA) aims to extract aspects from text-\nimage pairs and recognize their sentiments. Ex-\nisting methods make great efforts to align the\nwhole image to corresponding aspects. How-\never, different regions of the image may relate\nto different aspects in the same sentence, and\ncoarsely establishing image-aspect alignment\nwill introduce noise to aspect-based sentiment\nanalysis (i.e., visual noise). Besides, the sen-\ntiment of a specific aspect can also be inter-\nfered by descriptions of other aspects (i.e., tex-\ntual noise). Considering the aforementioned\nn

### Integration Vectordb Context pipeline With LLM output

In [ ]:
answer=rag_simple("What is attention mechanism?",rag_retriever,llm)
print(answer)

Retrieving documents for query: 'What is attention mechanism?'
Top K: 3, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generated embeddings with shape: (1, 384)
Retrieved 0 documents (after filtering)
No relevant context found to answer the question.


In [ ]:
### Simple RAG pipeline with Groq LLM
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv

# Load environment variables (make sure GROQ_API_KEY is in your .env file)
load_dotenv()
groq_api_key = os.environ.get("GROQ_API_KEY")

# Initialize the Groq LLM
llm = ChatGroq(
    groq_api_key=groq_api_key,
    model_name="gemma2-9b-it",
    temperature=0.1,
    max_tokens=1024
)

## Simple RAG function: retrieve context + generate response
def rag_simple(query, retriever, llm, top_k=3):
    # retrieve relevant docs
    results = retriever.retrieve(query, top_k=top_k)
    context = "\n\n".join([doc['content'] for doc in results]) if results else ""
    if not context:
        return "No relevant context found to answer the question."

    # build prompt safely
    prompt = f"""Use the following context to answer the question concisely.

Context:
{context}

Question: {query}

Answer:"""

    # generate response
    response = llm.invoke(prompt)
    return response.content

In [ ]:
# Example usage (rag_retriever must already be defined)
answer = rag_simple("What is multimodal aspect-based sentiment analysis?", rag_retriever, llm)
print(answer)

Retrieving documents for query: 'What is multimodal aspect-based sentiment analysis?'
Top K: 3, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generated embeddings with shape: (1, 384)
Retrieved 3 documents (after filtering)
Multimodal aspect-based sentiment analysis (MABSA) aims to extract aspects from text-image pairs and determine the sentiment associated with each aspect.  



In [ ]:
# Example usage (rag_retriever must already be defined)
answer = rag_simple("What the archetecture applayed in multimodal aspect-based sentiment analysis?", rag_retriever, llm)
print(answer)

Retrieving documents for query: 'What the archetecture applayed in multimodal aspect-based sentiment analysis?'
Top K: 3, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generated embeddings with shape: (1, 384)
Retrieved 3 documents (after filtering)
The paper proposes an **Aspect-oriented model (AoM)**. 

It uses two modules:

* **Aspect-Aware Attention Module:**  To align visual information and descriptions to corresponding aspects.
*  A module to detect aspect-relevant sentiment information. 


Let me know if you'd like more details about either module! 



### Enhanced RAG Pipeline Features

In [ ]:
# --- Enhanced RAG Pipeline Features ---
def rag_advanced(query, retriever, llm, top_k=5, min_score=0.2, return_context=False):
    """
    RAG pipeline with extra features:
    - Returns answer, sources, confidence score, and optionally full context.
    """
    results = retriever.retrieve(query, top_k=top_k, score_threshold=min_score)
    if not results:
        return {'answer': 'No relevant context found.', 'sources': [], 'confidence': 0.0, 'context': ''}

    # Prepare context and sources
    context = "\n\n".join([doc['content'] for doc in results])
    sources = [{
        'source': doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
        'page': doc['metadata'].get('page', 'unknown'),
        'score': doc['similarity_score'],
        'preview': doc['content'][:300] + '...'
    } for doc in results]
    confidence = max([doc['similarity_score'] for doc in results])

    # Generate answer
    prompt = f"""Use the following context to answer the question concisely.\nContext:\n{context}\n\nQuestion: {query}\n\nAnswer:"""
    response = llm.invoke([prompt.format(context=context, query=query)])

    output = {
        'answer': response.content,
        'sources': sources,
        'confidence': confidence
    }
    if return_context:
        output['context'] = context
    return output

# Example usage:
result = rag_advanced("What names of datasets Twitter-2015 used in multimoda aspect-based sentiment analysis", rag_retriever, llm, top_k=3, min_score=0.1, return_context=True)
print("Answer:", result['answer'])
print("Sources:", result['sources'])
print("Confidence:", result['confidence'])
print("Context Preview:", result['context'][:300])

Retrieving documents for query: 'What names of datasets Twitter-2015 used in multimoda aspect-based sentiment analysis'
Top K: 3, Score threshold: 0.1
Generating embeddings for 1 texts...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generated embeddings with shape: (1, 384)
Retrieved 3 documents (after filtering)
Answer: The provided text does not mention any datasets used in the context of multimoda aspect-based sentiment analysis. 



Sources: [{'source': 'AoM.pdf', 'page': 8, 'score': 0.24316471815109253, 'preview': 'cation. arXiv preprint arXiv:1906.03820.\nXincheng Ju, Dong Zhang, Rong Xiao, Junhui Li,\nShoushan Li, Min Zhang, and Guodong Zhou.\n2021. Joint Multi-modal Aspect-Sentiment Anal-\nysis with Auxiliary Cross-modal Relation Detec-\ntion. In Proceedings of the 2021 Conference on\nEmpirical Methods in Natural...'}, {'source': 'AoM.pdf', 'page': 8, 'score': 0.24316471815109253, 'preview': 'cation. arXiv preprint arXiv:1906.03820.\nXincheng Ju, Dong Zhang, Rong Xiao, Junhui Li,\nShoushan Li, Min Zhang, and Guodong Zhou.\n2021. Joint Multi-modal Aspect-Sentiment Anal-\nysis with Auxiliary Cross-modal Relation Detec-\ntion. In Proceedings of the 2021 Conference on\nEmpirical Methods in Natural...'}, {'sour

In [ ]:
# --- Advanced RAG Pipeline: Streaming, Citations, History, Summarization ---
from typing import List, Dict, Any
import time

class AdvancedRAGPipeline:
    def __init__(self, retriever, llm):
        self.retriever = retriever
        self.llm = llm
        self.history = []  # Store query history

    def query(self, question: str, top_k: int = 5, min_score: float = 0.2, stream: bool = False, summarize: bool = False) -> Dict[str, Any]:
        # Retrieve relevant documents
        results = self.retriever.retrieve(question, top_k=top_k, score_threshold=min_score)
        if not results:
            answer = "No relevant context found."
            sources = []
            context = ""
        else:
            context = "\n\n".join([doc['content'] for doc in results])
            sources = [{
                'source': doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
                'page': doc['metadata'].get('page', 'unknown'),
                'score': doc['similarity_score'],
                'preview': doc['content'][:120] + '...'
            } for doc in results]
            # Streaming answer simulation
            prompt = f"""Use the following context to answer the question concisely.\nContext:\n{context}\n\nQuestion: {question}\n\nAnswer:"""
            if stream:
                print("Streaming answer:")
                for i in range(0, len(prompt), 80):
                    print(prompt[i:i+80], end='', flush=True)
                    time.sleep(0.05)
                print()
            response = self.llm.invoke([prompt.format(context=context, question=question)])
            answer = response.content

        # Add citations to answer
        citations = [f"[{i+1}] {src['source']} (page {src['page']})" for i, src in enumerate(sources)]
        answer_with_citations = answer + "\n\nCitations:\n" + "\n".join(citations) if citations else answer

        # Optionally summarize answer
        summary = None
        if summarize and answer:
            summary_prompt = f"Summarize the following answer in 2 sentences:\n{answer}"
            summary_resp = self.llm.invoke([summary_prompt])
            summary = summary_resp.content

        # Store query history
        self.history.append({
            'question': question,
            'answer': answer,
            'sources': sources,
            'summary': summary
        })

        return {
            'question': question,
            'answer': answer_with_citations,
            'sources': sources,
            'summary': summary,
            'history': self.history
        }

# Example usage:
adv_rag = AdvancedRAGPipeline(rag_retriever, llm)
result = adv_rag.query("what is multimoda aspect-based sentiment analysis", top_k=3, min_score=0.1, stream=True, summarize=True)
print("\nFinal Answer:", result['answer'])
print("Summary:", result['summary'])
print("History:", result['history'][-1])

Retrieving documents for query: 'what is multimoda aspect-based sentiment analysis'
Top K: 3, Score threshold: 0.1
Generating embeddings for 1 texts...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generated embeddings with shape: (1, 384)
Retrieved 3 documents (after filtering)
Streaming answer:
Use the following context to answer the question concisely.
Context:
prediction.
5 Conclusion
In this paper, we proposed an aspect-oriented
model (AoM) for the task of multimodal aspect-
based sentiment analysis. We use two specially
designed modules to detect aspect-relevant infor-
mation from the semantic and sentiment perspec-
tives. On the one hand, to learn aspect-relevant
semantic information especially from the image,
we construct the Aspect-Aware Attention Module
to align the visual information and descriptions to
the corresponding aspect. On the other hand, to
detect the aspect-relevant sentiment information,

prediction.
5 Conclusion
In this paper, we proposed an aspect-oriented
model (AoM) for the task of multimodal aspect-
based sentiment analysis. We use two specially
designed modules to detect aspect-relevant infor-
mation from the semantic and sentiment perspec-
tives. On 

KeyError: 'zhouru,guowenya,liuxumeng,yushenglong,zhangying'